# **COVID-19 DASHBOARD**

Welcome to the Covid-19 dashboard. This dashboard downloads and displays Covid data from the Public Health England website. 

In [334]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API
import json

In [335]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [336]:
filters = [
    'areaType=overview'
]
structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",
    "deaths": "cumDeaths28DaysByDeathDateRate"    
}
api = Cov19API(filters=filters, structure=structure)
timeseries=api.get_json()
import json
with open("timeseries.json", "wt") as OUTF:
    json.dump(timeseries, OUTF)

In [337]:
with open("timeseries.json", "rt") as INFILE:
    data=json.load(INFILE)

In [338]:
datalist=data['data']
dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()

In [339]:
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [340]:
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])
print (startdate, ' to ', enddate)

2020-01-03 00:00:00  to  2020-11-26 00:00:00


In [341]:
index=pd.date_range(startdate, enddate, freq='D')
timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths'])

In [342]:
for entry in datalist: 
    date=parse_date(entry['date'])
    for column in ['cases', 'hospital', 'deaths']:
        if pd.isna(timeseriesdf.loc[date, column]): 
            value= float(entry[column]) if entry[column]!=None else 0.0
            timeseriesdf.loc[date, column]=value
            

timeseriesdf.fillna(0.0, inplace=True)

In [343]:
timeseriesdf.to_pickle("timeseriesdf.pkl")

In [344]:
def access_api():
    filters = [
    'areaType=overview'
    ]
    structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",
    "deaths": "cumDeaths28DaysByDeathDateRate"    
    }
    api = Cov19API(filters=filters, structure=structure)
    timeseries=api.get_json()
    return


def api_button_callback(button):
    apidata=access_api()
    global df
    timeseriesdf=wrangle_data(apidata)
    refresh_graph
    apibutton.icon="check"

apibutton=wdg.Button(
    description='Refresh Data', 
    disabled=False,
    button_style='success', 
    tooltip="Click to download current Public Health England data",
    icon='download'
)
    

apibutton.on_click(api_button_callback) 

display(apibutton)



Button(button_style='success', description='Refresh Data', icon='download', style=ButtonStyle(), tooltip='Clic…

## **Graph**

This graph displays the running statistics for Covid -19 cases, hospital admissions, and deaths. You can choose to view just one of these at a time, or view them all together. 

In [345]:
timeseriesdf=pd.read_pickle("timeseriesdf.pkl")

In [346]:
timeseriesdf.plot
    
series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)


def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})



ctrls=wdg.VBox([series, scale])

form=wdg.HBox([graph, ctrls])

display(form)
